In [1]:
# -*- coding: utf-8 -*-
"""
Enhanced Analysis (same directory, same random seed, same 90/10 split as the main experiment)
Outputs:
 - Uncertainty quantification: quantile interval coverage + prediction band figure
 - Generalization robustness: repeated K-fold distribution + validation curve
 - Error profiling: MAE by predicted/true quantiles
 - Interpretability: Permutation Importance + stability + PDP/ICE + 2D interactions
 - Data shift: PSI (Train vs Test) on top numeric features
 - Reusable tables: prediction details, error buckets, PI, PSI, etc. (CSV)
"""

import os, json, time, warnings
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import RepeatedKFold, validation_curve, train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.inspection import PartialDependenceDisplay, permutation_importance

# -------------------- Global plotting / fonts --------------------
sns.set_style("whitegrid")
plt.rcParams["axes.unicode_minus"] = False
warnings.filterwarnings("ignore", message=r"Glyph \d+ .* missing from font", category=UserWarning)

import matplotlib.font_manager as fm
FONT_CHAIN = [
    "Microsoft YaHei","Noto Sans CJK SC","SimHei","PingFang SC","Heiti SC",
    "Arial Unicode MS","DejaVu Sans"
]
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = FONT_CHAIN

# -------------------- Paths / constants --------------------
ROOT = Path.cwd()
REPORT_DIR = ROOT / "报告第一版"  # keep path unchanged as requested
REPORT_DIR.mkdir(parents=True, exist_ok=True)

DATA_PATH = ROOT / "Book1_5x_noise20-3.csv"  # keep the same as main experiment; change here if needed
TARGET = "average_renewable_energy_usage_percent"
RS = 42

print(f"\n📂 Data file: {DATA_PATH}")
print(f"📥 Output dir: {REPORT_DIR}\n")

t0 = time.time()

# -------------------- Read & preprocess (aligned with main experiment) --------------------
if not DATA_PATH.exists():
    raise FileNotFoundError(f"Data file not found: {DATA_PATH}")

df_raw = pd.read_csv(DATA_PATH)
print(f"✅ Loaded: {df_raw.shape[0]} rows, {df_raw.shape[1]} columns")

# Drop extremely sparse columns
missing_ratio = df_raw.isna().mean(numeric_only=False)
high_missing_cols = missing_ratio[missing_ratio >= 0.95].index.tolist()
if high_missing_cols:
    df_raw = df_raw.drop(columns=high_missing_cols)
    print(f"🧹 Dropped columns with ≥95% missing: {len(high_missing_cols)}")

# One-Hot encoding
obj_cols = df_raw.select_dtypes(include=["object"]).columns.tolist()
if TARGET not in df_raw.columns:
    raise ValueError(f"❌ Target column {TARGET} not found in raw data")
y_raw = pd.to_numeric(df_raw[TARGET], errors="coerce")
if y_raw.isna().any():
    y_raw = y_raw.fillna(np.nanmedian(y_raw))

df_oh = pd.get_dummies(df_raw, columns=obj_cols, drop_first=True)
y = pd.to_numeric(df_oh[TARGET], errors="coerce").fillna(np.nanmedian(y_raw))
X = df_oh.drop(columns=[TARGET]).apply(pd.to_numeric, errors="coerce")
X = X.fillna(X.median(numeric_only=True))
all_nan_cols = X.columns[X.isna().all()].tolist()
if all_nan_cols:
    X = X.drop(columns=all_nan_cols)

print(f"🎯 Features: {X.shape[1]} | Samples: {len(X)}")

# -------------------- Split (reuse saved indices from main experiment if present) --------------------
split_file = REPORT_DIR / f"split_indices_90_10_rs{RS}.npz"
if split_file.exists():
    test_idx = np.load(split_file)["test_idx"]
    test_idx = pd.Index(test_idx)
    all_idx = X.index
    train_idx = all_idx.difference(test_idx)
    X_train, X_test = X.loc[train_idx], X.loc[test_idx]
    y_train, y_test = y.loc[train_idx], y.loc[test_idx]
    print("🔁 Reused main-experiment split (90/10, same seed)")
else:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.10, random_state=RS, shuffle=True
    )
    print("✂️ No saved split found; created a fresh 90/10 split")

print(f"📊 Train: {len(X_train)} | Test: {len(X_test)}")

# -------------------- Train the same GB model --------------------
base_params = dict(n_estimators=500, learning_rate=0.05, max_depth=4, random_state=RS)
model = GradientBoostingRegressor(**base_params)
print("\n🚀 Training Gradient Boosting (same as main)…")
t1 = time.time()
model.fit(X_train, y_train)
print(f"⏱️ Fit time: {time.time()-t1:.2f}s")

y_pred = model.predict(X_test)
R2  = r2_score(y_test, y_pred)
RMSE = float(np.sqrt(mean_squared_error(y_test, y_pred)))
MAE  = float(mean_absolute_error(y_test, y_pred))
print(f"🏆 Test: R²={R2:.3f} | RMSE={RMSE:.2f} | MAE={MAE:.2f}")

# Save prediction details
pred_df = pd.DataFrame({
    "y_true": y_test.values,
    "y_pred": y_pred,
    "error": y_test.values - y_pred,
    "abs_error": np.abs(y_test.values - y_pred),
}, index=y_test.index)
pred_path = REPORT_DIR / "GB_pred_detail.csv"
pred_df.to_csv(pred_path, index=True, encoding="utf-8-sig")

# -------------------- A. Uncertainty via quantile GB (robust error bars) --------------------
print("\n[A] Training quantile GB to estimate 90% prediction intervals")
gb_lower = GradientBoostingRegressor(loss="quantile", alpha=0.05, **base_params)
gb_upper = GradientBoostingRegressor(loss="quantile", alpha=0.95, **base_params)
gb_lower.fit(X_train, y_train)
gb_upper.fit(X_train, y_train)
y_lo = gb_lower.predict(X_test)
y_hi = gb_upper.predict(X_test)

# Fix crossing + non-negative yerr + recompute coverage
ylo = np.minimum(y_lo, y_hi)
yhi = np.maximum(y_lo, y_hi)
coverage = float(np.mean((y_test.to_numpy() >= ylo) & (y_test.to_numpy() <= yhi)))
band_width = float(np.mean(yhi - ylo))

lower_err = np.maximum(0.0, (y_pred - ylo))
upper_err = np.maximum(0.0, (yhi - y_pred))
lower_err = np.nan_to_num(lower_err, nan=0.0, posinf=0.0, neginf=0.0)
upper_err = np.nan_to_num(upper_err, nan=0.0, posinf=0.0, neginf=0.0)

plt.figure(figsize=(9, 8))
plt.scatter(y_test, y_pred, alpha=0.7, label="Prediction")
lo_axis = float(min(y_test.min(), y_pred.min()))
hi_axis = float(max(y_test.max(), y_pred.max()))
plt.plot([lo_axis, hi_axis], [lo_axis, hi_axis], "r--", lw=2, label="y = x")

yerr = np.vstack([lower_err, upper_err])  # (2, N) asymmetric errors
plt.errorbar(
    y_test.to_numpy(), y_pred, yerr=yerr,
    fmt="none", alpha=0.25, capsize=2, errorevery=2, label="90% interval"
)

plt.title(f"Prediction vs. 90% Interval (coverage={coverage:.2%}, avg width={band_width:.2f})")
plt.xlabel("Actual"); plt.ylabel("Predicted")
plt.legend()
plt.tight_layout()
plt.savefig(REPORT_DIR / "A1_pred_with_PI.png", dpi=300, bbox_inches="tight")
plt.close()

pd.DataFrame([{"PI_coverage@90%": coverage, "avg_band_width": band_width}]).to_csv(
    REPORT_DIR / "GB_PI_coverage.csv", index=False, encoding="utf-8-sig"
)

# -------------------- B. Generalization: CV distribution & validation curve --------------------
print("\n[B] Repeated K-Fold CV + validation curve")
rkf = RepeatedKFold(n_splits=5, n_repeats=5, random_state=RS)
r2_list, rmse_list, mae_list = [], [], []
for tr_idx, va_idx in rkf.split(X, y):
    Xtr, Xva = X.iloc[tr_idx], X.iloc[va_idx]
    ytr, yva = y.iloc[tr_idx], y.iloc[va_idx]
    m = GradientBoostingRegressor(**base_params).fit(Xtr, ytr)
    yp = m.predict(Xva)
    r2_list.append(r2_score(yva, yp))
    rmse_list.append(np.sqrt(mean_squared_error(yva, yp)))
    mae_list.append(mean_absolute_error(yva, yp))

cv_df = pd.DataFrame({"R2": r2_list, "RMSE": rmse_list, "MAE": mae_list})
cv_df.to_csv(REPORT_DIR/"B1_CV_distribution.csv", index=False, encoding="utf-8-sig")

plt.figure(figsize=(10,6))
sns.violinplot(data=cv_df[["R2","RMSE","MAE"]], inner="quartile")
plt.title("Repeated K-Fold (5×5) Performance Distribution")
plt.tight_layout()
plt.savefig(REPORT_DIR/"B1_CV_violin.png", dpi=300)
plt.close()

param_range = np.array([100, 200, 300, 500, 800])
train_scores, valid_scores = validation_curve(
    GradientBoostingRegressor(learning_rate=0.05, max_depth=4, random_state=RS),
    X, y, param_name="n_estimators", param_range=param_range, scoring="r2", cv=5
)
plt.figure(figsize=(10,6))
plt.plot(param_range, train_scores.mean(axis=1), "o-", label="Train R²")
plt.plot(param_range, valid_scores.mean(axis=1), "o-", label="Validation R²")
plt.fill_between(param_range, valid_scores.mean(1)-valid_scores.std(1),
                 valid_scores.mean(1)+valid_scores.std(1), alpha=0.2)
plt.xlabel("n_estimators"); plt.ylabel("R²"); plt.title("Validation Curve: n_estimators")
plt.legend()
plt.tight_layout()
plt.savefig(REPORT_DIR/"B2_validation_curve_n_estimators.png", dpi=300)
plt.close()

# -------------------- C. Error profiling: quantile buckets --------------------
print("\n[C] Error by quantile buckets")
def bucket_mae(series_ref, errors, q=10, label="pred"):
    qcuts = pd.qcut(series_ref, q=q, duplicates="drop")
    # key: observed=True to silence future pandas warning and keep current behavior
    grp = (pd.DataFrame({"bucket": qcuts, "abs_error": np.abs(errors)})
           .groupby("bucket", observed=True)["abs_error"])
    out = grp.agg(["count","mean"]).reset_index().rename(columns={"mean": f"MAE_by_{label}"})
    return out

err = y_test.values - y_pred
b1 = bucket_mae(pd.Series(y_pred, index=y_test.index), err, q=10, label="pred")
b2 = bucket_mae(pd.Series(y_test.values, index=y_test.index), err, q=10, label="true")
b1.to_csv(REPORT_DIR/"C1_bucket_by_pred.csv", index=False, encoding="utf-8-sig")
b2.to_csv(REPORT_DIR/"C2_bucket_by_true.csv", index=False, encoding="utf-8-sig")

plt.figure(figsize=(10,6))
plt.plot(np.arange(len(b1))+1, b1["MAE_by_pred"], "o-")
plt.title("MAE by Predicted Quantiles (harder to the right)")
plt.xlabel("Predicted quantile bucket (1 = lowest)"); plt.ylabel("MAE")
plt.tight_layout()
plt.savefig(REPORT_DIR/"C1_MAE_by_pred_quantile.png", dpi=300)
plt.close()

plt.figure(figsize=(10,6))
plt.plot(np.arange(len(b2))+1, b2["MAE_by_true"], "o-")
plt.title("MAE by True Quantiles")
plt.xlabel("True quantile bucket (1 = lowest)"); plt.ylabel("MAE")
plt.tight_layout()
plt.savefig(REPORT_DIR/"C2_MAE_by_true_quantile.png", dpi=300)
plt.close()

# -------------------- D. Interpretability: PI (+ stability) --------------------
print("\n[D] Permutation Importance (PI) + stability")
pi = permutation_importance(model, X_test, y_test, n_repeats=10, scoring="r2", random_state=RS)
pi_df = pd.DataFrame({
    "Feature": X_test.columns,
    "PI_mean": pi.importances_mean,
    "PI_std": pi.importances_std
}).sort_values("PI_mean", ascending=False)
pi_df.to_csv(REPORT_DIR/"D1_PI_test.csv", index=False, encoding="utf-8-sig")

topk = min(15, len(pi_df))
top_df = pi_df.head(topk).iloc[::-1]
plt.figure(figsize=(max(11, 0.45*topk+9), 7))
plt.barh(top_df["Feature"], top_df["PI_mean"], xerr=top_df["PI_std"], alpha=0.8)
plt.title(f"Permutation Importance (Top {topk}, test set)")
plt.xlabel("importance (mean ± std over permutations)"); plt.ylabel("Feature")
plt.tight_layout()
plt.savefig(REPORT_DIR/"D1_PI_barh.png", dpi=300)
plt.close()

# Stability: bootstrap ranking fluctuation
boot = 15
ranks = []
rng = np.random.default_rng(RS)
test_idx_arr = X_test.index.to_numpy()
for b in range(boot):
    sub_idx = rng.choice(test_idx_arr, size=len(test_idx_arr), replace=True)
    res = permutation_importance(model, X_test.loc[sub_idx], y_test.loc[sub_idx],
                                 n_repeats=5, scoring="r2", random_state=RS+b)
    mean_imp = pd.Series(res.importances_mean, index=X_test.columns).sort_values(ascending=False)
    ranks.append(mean_imp.rank(ascending=False, method="min"))
rank_df = pd.concat(ranks, axis=1)
rank_stats = pd.DataFrame({
    "Feature": rank_df.index,
    "rank_mean": rank_df.mean(1),
    "rank_std": rank_df.std(1)
}).sort_values("rank_mean").head(topk)
rank_stats.to_csv(REPORT_DIR/"D2_PI_rank_stability.csv", index=False, encoding="utf-8-sig")

plt.figure(figsize=(max(11, 0.45*topk+9), 7))
plt.errorbar(rank_stats["rank_mean"], np.arange(len(rank_stats)), xerr=rank_stats["rank_std"],
             fmt="o", capsize=3)
plt.yticks(np.arange(len(rank_stats)), rank_stats["Feature"])
plt.gca().invert_yaxis()
plt.xlabel("Average rank (smaller = more important) ± std"); plt.title("PI Rank Stability (bootstrap 15×)")
plt.tight_layout()
plt.savefig(REPORT_DIR/"D2_PI_rank_stability.png", dpi=300)
plt.close()

# -------------------- E. PDP + ICE + 2D interactions --------------------
print("\n[E] PDP/ICE/2D interactions")
preferred = ["internet_penetration_percent","power_capacity_MW_total","total_data_centers"]
fi = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()

use_feats = [c for c in preferred if c in X.columns][:3]
for ftr in fi.index:
    if len(use_feats) >= 3: break
    if (ftr in numeric_cols) and (ftr not in use_feats):
        use_feats.append(ftr)

# PDP + ICE
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
try:
    PartialDependenceDisplay.from_estimator(model, X, use_feats, ax=ax, kind="both", subsample=200)
    fig.suptitle("PDP + ICE (subsample=200)", fontsize=14)
except Exception:
    plt.close(fig)
    fig, ax = plt.subplots(figsize=(8,3))
    ax.axis("off"); ax.text(0.02,0.6,f"PDP/ICE generation failed, features: {use_feats}", fontsize=12)
plt.tight_layout(); plt.savefig(REPORT_DIR/"E1_PDP_ICE.png", dpi=300); plt.close()

# 2D interaction (top two numeric features)
pair = []
for f in fi.index:
    if f in numeric_cols:
        pair.append(f)
    if len(pair) >= 2:
        break

fig = plt.figure(figsize=(6,5))
try:
    PartialDependenceDisplay.from_estimator(model, X, [tuple(pair)], kind="average")
    plt.title(f"2D Interaction PDP: {pair[0]} × {pair[1]}")
except Exception:
    plt.close(fig)
    fig, ax = plt.subplots(figsize=(8,3))
    ax.axis("off"); ax.text(0.02,0.6, f"2D PDP generation failed, features: {pair}", fontsize=12)
plt.tight_layout(); plt.savefig(REPORT_DIR/"E2_PDP_2D.png", dpi=300); plt.close()

# -------------------- F. Data shift: PSI (Train vs Test) --------------------
print("\n[F] PSI: Train vs Test distribution difference (top numeric features)")
def compute_psi(a, b, bins=10, eps=1e-9):
    # Bins are based on 'a' (train)
    quantiles = np.linspace(0,1,bins+1)
    edges = np.unique(np.quantile(a, quantiles))
    # Guard against all-equal values
    if len(edges) < 3:
        edges = np.linspace(min(a.min(), b.min()), max(a.max(), b.max()), bins+1)
    a_cnt, _ = np.histogram(a, bins=edges)
    b_cnt, _ = np.histogram(b, bins=edges)
    a_pct = a_cnt / max(a_cnt.sum(), 1)
    b_pct = b_cnt / max(b_cnt.sum(), 1)
    a_pct = np.clip(a_pct, eps, None)
    b_pct = np.clip(b_pct, eps, None)
    psi = np.sum((a_pct - b_pct) * np.log(a_pct / b_pct))
    return float(psi), edges

top_for_psi = [f for f in fi.index if f in numeric_cols][:5]
psi_rows = []
for f in top_for_psi:
    v_tr, v_te = X_train[f].values, X_test[f].values
    val, edges = compute_psi(v_tr, v_te, bins=10)
    psi_rows.append({"Feature": f, "PSI": val, "Bins": len(edges)-1})
psi_df = pd.DataFrame(psi_rows).sort_values("PSI", ascending=False)
psi_df.to_csv(REPORT_DIR/"F1_PSI_train_vs_test.csv", index=False, encoding="utf-8-sig")

plt.figure(figsize=(8,5))
plt.barh(psi_df["Feature"][::-1], psi_df["PSI"][::-1])
plt.xlabel("PSI (>0.2 monitor, >0.5 strong shift)"); plt.title("Train vs Test PSI (top numeric features)")
plt.tight_layout(); plt.savefig(REPORT_DIR/"F1_PSI_barh.png", dpi=300); plt.close()

# -------------------- Done & artifact list --------------------
artifacts = [
    "GB_pred_detail.csv",
    "GB_PI_coverage.csv",
    "B1_CV_distribution.csv",
    "B1_CV_violin.png",
    "B2_validation_curve_n_estimators.png",
    "C1_bucket_by_pred.csv",
    "C2_bucket_by_true.csv",
    "C1_MAE_by_pred_quantile.png",
    "C2_MAE_by_true_quantile.png",
    "D1_PI_test.csv",
    "D1_PI_barh.png",
    "D2_PI_rank_stability.csv",
    "D2_PI_rank_stability.png",
    "E1_PDP_ICE.png",
    "E2_PDP_2D.png",
    "F1_PSI_train_vs_test.csv",
    "F1_PSI_barh.png",
    "A1_pred_with_PI.png",
]
print("\n✅ Enhanced analysis completed. Artifacts:")
for a in artifacts:
    p = REPORT_DIR / a
    if p.exists():
        print(" -", p)
    else:
        print(" - (missing)", p)

print(f"\n⏱️ Total time: {time.time()-t0:.2f}s")



📂 Data file: C:\Users\hejin\DM_FINALVERSION_ENG\Book1_5x_noise20-3.csv
📥 Output dir: C:\Users\hejin\DM_FINALVERSION_ENG\报告第一版

✅ Loaded: 955 rows, 168 columns
🧹 Dropped columns with ≥95% missing: 1
🎯 Features: 1600 | Samples: 955
✂️ No saved split found; created a fresh 90/10 split
📊 Train: 859 | Test: 96

🚀 Training Gradient Boosting (same as main)…
⏱️ Fit time: 14.71s
🏆 Test: R²=0.955 | RMSE=5.99 | MAE=3.92

[A] Training quantile GB to estimate 90% prediction intervals

[B] Repeated K-Fold CV + validation curve

[C] Error by quantile buckets

[D] Permutation Importance (PI) + stability

[E] PDP/ICE/2D interactions

[F] PSI: Train vs Test distribution difference (top numeric features)

✅ Enhanced analysis completed. Artifacts:
 - C:\Users\hejin\DM_FINALVERSION_ENG\报告第一版\GB_pred_detail.csv
 - C:\Users\hejin\DM_FINALVERSION_ENG\报告第一版\GB_PI_coverage.csv
 - C:\Users\hejin\DM_FINALVERSION_ENG\报告第一版\B1_CV_distribution.csv
 - C:\Users\hejin\DM_FINALVERSION_ENG\报告第一版\B1_CV_violin.png
 - C:\U

<Figure size 600x500 with 0 Axes>